# 1. 언어 모델

## 1-1. 언어 모델이란?

* **정의:** 단어 시퀀스(문장)에 확률을 할당, 가장 자연스러운 단어 시퀀스를 찾아내는 모델.
* **보편적 훈련법:** '이전 단어들'이 주어졌을 때 '다음 단어'를 예측하도록 훈련.
* **활용 예시:**
    * **기계 번역:** P(나는 버스를 탔다) > P(나는 버스를 태운다) (더 자연스러운 문장 판단)
    * **오타 교정:** P(달려갔다) > P(잘려갔다) (더 적절한 문장 판단)
* **주요 구분:**
    * **통계 기반:** 문장 등장 빈도 기반 확률 계산. (복잡도 낮음, 예: N-gram)
    * **인공 신경망 기반:** 단어 벡터화, 신경망으로 다음 단어 예측. (복잡도 높음, 예: RNN, Transformer)

---

## 1-2. SLM(통계적 언어 모델)

### (1) Statistic Language Model이란?

* **핵심 근거:** **분포 가설 (Distributional Hypothesis)**
    * **정의:** "비슷한 문맥에서 함께 나타나는 단어들은 비슷한 의미를 가진다."
    * **예시:** "I eat \_\_\_\_" 문맥에 'pizza', 'chicken', 'pasta'가 공통 등장 → '음식'이라는 유사 카테고리로 학습.
* **기본 개념:** 조건부 확률, 카운트 기반 확률.

#### 1️⃣ 조건부 확률

* **정의:** A가 일어났다는 전제 하에 B가 일어날 확률.
* **문장 확률 계산:** 다음 단어를 순차적으로 예측하는 확률을 모두 곱함.
    * **예시:** $P(\text{오늘}, \text{날씨가}, \text{좋다}) = P(\text{오늘}) \times P(\text{날씨가}|\text{오늘}) \times P(\text{좋다}|\text{오늘}, \text{날씨가})$
* **결론:** 확률값이 높을수록 모델은 해당 문장을 더 자연스럽다고 판단.

#### 2️⃣ 카운트 기반

* **정의:** 이전 단어 시퀀스의 등장 빈도(카운트)를 통해 다음 단어의 확률을 계산.
* **계산 예시:** P(좋다|오늘 날씨가) = ('오늘 날씨가 좋다' 등장 횟수) / ('오늘 날씨가' 등장 총 횟수)
* **한계:** **희소 문제 (Sparsity Problem)**
    * **정의:** 한정된 학습 데이터로 인해 현실의 무한한 언어를 모델링하지 못하는 문제.
    * **상황 1 (분자 0):** 학습 데이터에 '오늘 날씨가 좋다'가 없으면, 확률 0%로 계산. (자연스러운데도 불가능으로 판단)
    * **상황 2 (분모 0):** '오늘 날씨가' 자체가 없으면, 확률 계산 불가.
    * **결론:** 이 한계로 인해 인공 신경망 기반 모델로 패러다임이 전환됨.

### (2) N-gram

* **등장 배경:** 기존 SLM의 **희소 문제**를 완화하기 위해, 참고하는 단어의 범위를 n개로 제한.
    * **아이디어:** P(좋다 | 오늘 날씨가) 대신 P(좋다 | 날씨가)를 계산. (참고 단어를 줄여 카운트 가능성 높임)
* **정의:** 연속된 n개의 단어 묶음(시퀀스).
    * 예: n=1 (unigram), n=2 (bigram), n=3 (trigram)
* **확률 계산:** 다음 단어 예측 시, 모든 앞 단어가 아닌 **(n-1)개의 단어**만 참고.
* **예측 방식 (3-gram 예시):** "어제 비가 많이 와서 오늘 하늘이 매우 \_\_\_?" 라는 문장에서 '___'를 예측할 때, 앞의 (n-1)개인 '하늘이 매우' 2개 단어만 참고.
* **N-gram의 한계:**
    * **(1) 여전한 희소 문제:** n을 줄여도 학습 데이터에 없는 n-gram은 여전히 확률 계산 불가.
    * **(2) n 선택의 Trade-off:**
        * **n (크게):** 문맥 정확도 증가, 희소 문제 심화.
        * **n (작게):** 희소 문제 완화, 문맥 파악 정확도 감소.
    * **(3) 문맥 파악의 한계:** n개 범위를 벗어난 장기 의존성 파악 불가. (예: "오늘 점심... 날씨가 좋다"에서 '날씨가'와 '좋다'의 연관성 파악 어려움)

### (3) Perplexity(PPL)

* **개념:** 언어 모델의 성능을 정량적으로 평가하기 위한 지표.
* **정의:** 언어 모델이 특정 문장을 얼마나 '혼란스러워(perplexed) 하는지'를 나타내는 수치.
* **핵심:** **PPL은 낮을수록 좋음.** (PPL 낮음 = 덜 혼란스러움 = 문맥 이해도 높음 = 성능 좋음)
* **수식:** 문장 W의 확률(P(W))에 역수를 취한 후, 문장의 단어 수(N)로 N제곱근.
    * P(W)가 높을수록 PPL은 낮아짐.
* **PPL 영향 요인:** 문장의 희소성(희소할수록 PPL 증가), 문장 길이(N)
* **해석 예시:** PPL이 10이면, 모델이 다음 단어 예측 시 평균 10개의 단어를 놓고 헷갈려 한다는 의미.
* **PPL을 낮추는 법:** 통계 모델은 희소 문제로 PPL이 높으나, 딥러닝 기반 모델은 일반화 능력이 뛰어나 PPL이 낮음.

---

## 1-3. 딥러닝 기반 언어모델

### (1) LLM(Large Language Model)

* **정의:** 대규모 언어 모델(LLM). 방대한 양의 데이터를 학습하여 인간 언어의 이해, 생성, 요약 등을 수행하는 AI 모델. (예: GPT-3, Gemini)
* **발전 과정:** SLM(Small) → NLM(Neural) → PLM(Pretrained) → LLM(Large)
* **LLM vs. AI 서비스:**
    * **LLM:** '천재의 뇌'. 복잡한 주문을 이해하고 레시피 지식을 가진 핵심 두뇌.
    * **AI 서비스:** LLM(뇌) + 도구(검색, 계산, 인터페이스 등)가 결합된 '만능 요리사 시스템' 전체.
* **작동 방식:**
    * **1. 대규모 학습 (Training):** 웹페이지, 책 등 방대한 데이터로 언어의 패턴, 문맥, 지식 학습.
    * **2. 예측 및 생성 (Prediction):** 사용자 지시(프롬프트)에 따라, 학습한 패턴을 기반으로 가장 확률 높은 다음 단어를 순차적 생성.
* **주요 사용 사례:**
    * **텍스트 생성:** 소설, 기사, 마케팅 콘텐츠 등.
    * **기계 번역:** 문맥과 뉘앙스를 고려한 번역.
    * **질의응답:** 의도 파악 및 정보 검색 기반 답변.
    * **문서 요약:** 긴 문서의 핵심 정보 요약.
    * **감정 분석:** 텍스트의 긍정/부정/중립 예측.

### (2) BERT

#### 1️⃣ BERT가 뭐예요?

* **개념:** **B**idirectional **E**ncoder **R**epresentations from **T**ransformers.
* **정의:** 트랜스포머의 **인코더(Encoder)** 부분을 활용해 만든 **양방향(Bidirectional)** 언어 모델.
* **핵심 강점:** "문장을 이해하는 능력"이 뛰어남. (예: Google 검색, 음성 비서, 스팸 메일 분류)
* **주요 특징:**
    * **양방향:** 한 단어(예: '한강')를 이해할 때, 그 단어의 좌우 문맥('나는 어제', '에서 자전거를')을 **동시에** 고려.
    * **트랜스포머 기반:** 트랜스포머 구조의 인코더 부분만 사용.
    * **사전 학습 + 파인 튜닝:** 대규모 데이터로 일반 언어 능력(사전학습)을 먼저 배우고, 특정 과제(감정 분석 등)에 맞게 미세 조정(파인 튜닝).

#### 2️⃣ 그래서 BERT는 어떻게 학습했나요?

* **구조:** 트랜스포머 인코더 12개(Base) 또는 24개(Large)를 쌓아 올린 구조.
* **임베딩:** 3가지 정보를 합쳐서 입력 벡터 생성.
    * **Token Embedding:** 단어 자체의 의미.
    * **Segment Embedding:** 두 문장 구분 (A 문장, B 문장).
    * **Position Embedding:** 단어의 위치 정보.
* **사전학습 (1): Masked Language Model (MLM)**
    * **개념:** "빈칸 채우기 퀴즈".
    * **방식:** 문장 속 단어 15%를 랜덤으로 [MASK] 처리하고, 주변 문맥(양방향)을 이용해 가려진 단어를 맞추도록 학습.
* **사전학습 (2): Next Sentence Prediction (NSP)**
    * **개념:** 두 문장이 실제로 이어지는 문장인지 아닌지 맞추는 학습.
    * **목적:** 문장 간의 관계 이해 능력 배양. (예: IsNext / NotNext 구분)

#### 3️⃣ BERT 이후의 모델은 무엇이 있나요?

* **RoBERTa:** BERT의 학습법(NSP 제거, 더 큰 데이터셋 등)을 개선하여 성능 향상.
* **ALBERT (A Lite BERT):** BERT의 경량화 버전. (파라미터 공유, 임베딩 행렬 분해)

### (3) GPT

#### 1️⃣ GPT는 뭐예요?

* **개념:** **G**enerative **P**re-trained **T**ransformer. (OpenAI 개발)
* **역사:**
    * **GPT-1:** 사전학습 + 태스크별 미세조정(지도학습)
    * **GPT-2:** 미세조정 단계 제거. 범용 언어모델 가능성 제시. (15억 파라미터)
    * **GPT-3:** Zero-shot, One-shot, Few-shot 및 Prompting 도입. (1750억 파라미터)
    * **GPT-4:** 멀티모달 기능 도입 (이미지, 텍스트 동시 처리).
* **아키텍처:** 트랜스포머의 **디코더(Decoder)**만 사용.
* **동작 원리:** **Autoregressive (단방향)**
    * 현재 단어 예측 시, **앞의 단어들(왼쪽 문맥)만** 참고. (Masked Self-Attention)
    * 다음에 올 단어의 확률을 계산하고 선택하며 순차적으로 문장 **생성**.
* **BERT와의 차이:**
    * **BERT:** 인코더, 양방향, 문맥 **이해** 중심. (빈칸 맞추기)
    * **GPT:** 디코더, 단방향, 문장 **생성** 중심. (다음 단어 예측)

#### 2️⃣ LLM의 한계

* **Hallucination (환각):** 사실과 다른 내용을 그럴듯하게 만들어내는 현상. (예: 세종대왕 맥북프로 던짐 사건)
* **업데이트 비용:** 새로운 정보 반영을 위한 재학습에 막대한 계산 자원과 비용 필요.

### (4) RAG

#### 1️⃣ RAG의 필요성

* **LLM의 한계:** 최신 정보나 특정 도메인 지식 부족, 재학습 비효율, 모르는 정보에 대한 환각 문제 발생.
* **RAG의 등장:** 이러한 한계를 극복하기 위해 등장.
* **RAG (Retrieval Augmented Generation) 정의:** 관련된 정보를 **검색(Retrieval)**하여 먼저 찾고, 이를 LLM에 **주입(Augmented)**하여 최종 답변을 **생성(Generation)**하는 파이프라인.
* **핵심 개념:** "모르면 (외부 문서를) 물어보자!"

#### 2️⃣ RAG의 구조

* **RAG 3단계:**
    * **1. 질의 인코더 (Query Encoder):** 사용자의 질문을 벡터 형태로 인코딩(변환).
    * **2. 지식 검색기 (Knowledge Retriever):** 인코딩된 질문 벡터를 바탕으로, 외부 지식 베이스(DB, Wikipedia 등)에서 관련성 높은 정보 검색.
    * **3. 지식 증강 생성기 (Generator):** (사용자 질문 + 검색된 지식)을 LLM에 함께 입력하여, 정보에 기반한 최종 답변 생성.

#### 3️⃣ RAG의 장점

* **풍부한 정보 제공:** 외부 데이터를 활용해 구체적이고 풍부한 답변 가능.
* **실시간 정보 반영:** DB에 최신 데이터만 추가하면, 모델 재학습 없이도 최신 정보 반영 가능.
* **환각 방지:** 실제 검색된 데이터에 기반하여 답변하므로, 환각 현상 감소 및 정확도 향상.

#### 4️⃣ CAG (Credibility-Aware Generation)

* **등장:** RAG의 한계(검색된 문서가 잘못되거나 신뢰성이 낮은 경우)를 보완.
* **개념:** 검색된 문서의 **신뢰도(Credibility)**를 모델이 스스로 판단하고 활용하도록 훈련시키는 프레임워크.

### (5) LangChain

#### 1️⃣ 그럼 랭체인은 뭐예요?

* **정의:** LLM을 기반으로 RAG와 같은 복잡한 애플리케이션(파이프라인)을 쉽게 구성할 수 있도록 도와주는 **프레임워크**.
* **비유:** LLM 앱을 빠르게 조립할 수 있는 "레고 블럭".

#### 2️⃣ LangChain (특징)

* **추상화 (Abstraction):** 문서 로드, 텍스트 분할, 임베딩, 벡터 DB 저장 등 복잡한 작업을 간결한 코드로 추상화.
* **표준화 (Standardization):** 다양한 AI 모델, 원본 데이터(Document) 등을 표준화된 컴포넌트 형식으로 통일.
* **체이닝 (Chaining):** 주요 컴포넌트들을 `|` (파이프) 연산자로 쉽게 연결하여 로직 구성. (예: `prompt | llm | parser`)

#### 3️⃣ LangGraph

* **LangChain의 한계:** 순차적(선형적) 작업 흐름에 적합. 조건부 분기, 순환(Loop) 등 복잡한 로직 구현 어려움.
* **LangGraph:**
    * **개념:** **비선형적** 작업 흐름 (Cycle, Loop) 구현 가능.
    * **구성:** **노드 (Node)**(작업 단위)와 **엣지 (Edge)**(연결선, 조건부 엣지)로 복잡한 로직 구현.
    * **활용:** 복잡한 자율 에이전트, "결과가 만족스러울 때까지 재시도" 같은 순환 로직에 적합.

#### 4️⃣ LangSmith

* **정의:** LLM 기반 애플리케이션을 위한 통합 **개발 및 운영 (DevOps) 플랫폼**.
* **등장 배경:** LLM 앱은 프롬프트부터 최종 답변까지의 중간 과정을 추적하고 디버깅하기 어려움.
* **핵심 기능:**
    * **디버깅 및 추적 (Tracing):** 모든 실행 과정을 단계별로 시각화.
    * **테스트 및 평가 (Evaluation):** 데이터셋을 이용해 앱 성능 자동 평가.
    * **모니터링 (Monitoring):** API 호출 횟수, 지연 시간, 토큰 사용량 등 실제 사용 현황 모니터링.
    * **프롬프트 허브 (Prompt Hub):** 프롬프트 저장, 버전 관리 및 테스트.

#### 5️⃣ 외부 API 활용하기

* **필요성:** LLM 내부 지식만으로 해결할 수 없는 질문(예: 실시간 날씨, 주식)은 외부 API 연동으로 해결.
* **OpenAI:** GPT, DALL-E 등을 개발한 AI 연구 기관.

---

## 1-4. sLM (small Language Model)

* **정의 (sLM):** 소규모 언어 모델. LLM보다 규모와 범위가 작은 AI 모델.
* **등장 배경:** LLM의 단점(높은 비용, 느린 속도, 데이터 보안 문제)을 보완. 경량화, 빠른 속도, 특정 작업 최적화에 중점.
* **sLM vs. LLM 비교:**
    * **매개변수:** sLM (수백만 ~ 수십억) << LLM (수천억 ~ 수조)
    * **sLM 장점:** 경량화, 빠른 학습/추론, 특정 작업 최적화, 적은 비용, 엣지/모바일 장치 및 오프라인 환경 유리.
    * **sLM 단점:** 제한된 성능과 범용성.

### (1) 모델 압축

* **1️⃣ 가지치기 (Pruning):** 신경망에서 중요도가 낮거나 불필요한 매개변수(가중치)를 제거하는 방법.
* **2️⃣ 양자화 (Quantization):** 고정밀 데이터(예: 32비트 부동소수점)를 저정밀 데이터(예: 8비트 정수)로 변환. (계산 부하 감소, 속도 향상)
* **3️⃣ 지식 증류 (Knowledge Distillation):** 사전 학습된 "교사 모델"(대형)의 지식을 "학생 모델"(소형)로 이전시켜 학습.

### (2) sLM의 예

* DistilBERT, Gemma, GPT-4o mini, Llama, Mistral, Phi 등.

### (3) sLM의 장점

* **프라이버시:** 외부 서버가 아닌 로컬 환경(온디바이스)에서 실행 가능하여 보안에 유리. (예: 기업 내부 문서 처리)
* **비용 절감:** 학습과 추론에 필요한 자원이 적어 비용과 에너지 소비가 낮음. (LLM 대비 10~30배 저렴)
* **효율성 & 맞춤화:** 특정 작업(예: 코딩, 한국어 서비스)에 최적화하기 쉬우며, 실시간 처리 등에서 높은 성능 보장.

### (4) sLM의 한계

* **주요 한계:** 편향 학습, 제한된 일반화(광범위한 지식 부족), 할루시네이션, LLM 대비 성능/용량 부족.
* **한계 극복 방안:**
    * **학습 단계:**
        * **(1) 미세조정/전문화 (Fine-Tuning):** 특정 도메인 데이터(법률, 회계 등)를 추가 학습 (예: LoRA).
        * **(2) 지식 증류 (KD):** '교사 LLM'의 고품질 답변을 '학생 sLM'이 학습.
    * **추론 단계:**
        * **(1) RAG (Retrieval-Augmented Generation):** sLM이 모르는 정보는 외부 DB에서 '검색(Retrieve)'하여 답변 생성. (오픈북 테스트 비유)
        * **(2) 결합 추론 (Hybrid Inference):** sLM과 LLM을 결합.
    * **용량 한계 극복:** 양자화, 가지치기.

### (5) LLM과 sLM의 결합 (결합 추론)

* **개념:** sLM이 어려운 문제를 풀 때, 막히는 부분만 LLM에게 선택적으로 도움받는 방식. (신입사원(sLM)이 선임자(LLM)에게 질문)
* **작동 원리:**
    * **1. sLM이 먼저 초안 작성:** sLM이 문제 풀이 초안 생성.
    * **2. 각 단계 평가/점수화:** 시스템이 각 단계의 신뢰도 점수화
    * **3. LLM이 어려운 부분만 수정:** 점수가 낮아 '불확실함'으로 판단된 단계만 LLM에게 요청하여 수정.
* **장점:** LLM 수준의 정확도를 달성하면서, LLM 토큰 사용량을 크게 절약 (최대 90%).

---

# 2. 생성 모델

## 2-1. 분류 모델 VS 생성 모델

* **생성 모델 (Generative Model):** 주어진 학습 데이터를 학습, 학습 데이터의 분포를 따르는 유사한 데이터를 생성하는 모델.
* **분류 모델 vs. 생성 모델:**
    * **분류 모델 (Discriminative):** 데이터 x가 어떤 레이블 y에 속하는지 판단. (P(y|x))
    * **생성 모델 (Generative):** 입력 데이터의 확률 분포 P(x) 자체를 학습하여 데이터를 생성.
* **생성 모델의 분류:**
    * **명시적 확률밀도 모델 (Explicit):** 확률 분포를 직접 표현/근사. (예: VAE, PixelRNN)
    * **암시적 확률밀도 모델 (Implicit):** 분포와 상관없이 생성. (예: GAN)

---

## 2-2. AE, VAE

### (1) AE (Auto Encoder)

* **정의:** 입력과 동일한 출력을 만드는 것을 목적으로 하는 신경망.
* **목적:** 차원 축소, 특징 추출, 노이즈 제거, 이상 탐지 등.
* **구조 (Diabolo Network):**
    * **인코더 (Encoder):** 고차원 입력($\mathbf{x}$)을 저차원 **잠재 벡터($\mathbf{z}$)**로 압축. (데이터의 '중요한' 정보 압축)
    * **디코더 (Decoder):** 잠재 벡터($\mathbf{z}$)를 다시 원래 차원으로 복원($\mathbf{x}'$). (압축 검증용)
* **학습 목표:** 입력 $\mathbf{x}$와 출력 $\mathbf{x}'$가 같아지도록 훈련.

### (2) VAE (Variational Auto Encoder)

* **개념:** **확률적 오토인코더(VAE)**. 랜덤 노이즈로부터 원하는 데이터를 생성하려는 목적.
* **목적:** 이미지 생성, 텍스트 생성 등 새로운 데이터 생성 작업.
* **AE vs. VAE:**
    * **목적:** AE (인코더 학습 - 차원 축소), VAE (디코더 학습 - 데이터 생성)
    * **잠재 벡터($\mathbf{z}$):**
        * **AE:** 잠재 코드가 **특정 값 (하나의 숫자)**.
        * **VAE:** 잠재 코드가 **확률 분포 (평균 $\mu$, 분산 $\sigma^2$)**.
    * **결과:** VAE가 잠재 공간을 더 잘 뭉치게(연속적으로) 표현하여 원본 데이터 재생 및 특징 파악에 유리.
* **VAE 구조:**
    * **인코더 ($q_\phi$):** 입력($\mathbf{x}$)을 단일 벡터가 아닌 **분포(평균 $\mu$, 표준편차 $\sigma$)**로 변환.
    * **샘플링 (Reparameterization trick):** 이 분포에서 $\mathbf{z}$를 샘플링.
    * **디코더 ($p_\theta$):** 샘플링된 잠재 벡터($\mathbf{z}$)로부터 입력($\mathbf{x}'$)을 복원(생성).

---

## 2-3. GAN (Generative Adversarial Network)

* **정의:** **생성적 적대 신경망(GAN)**. 생성자(Generator)와 판별자(Discriminator)가 서로 적대적으로 경쟁하며 학습하는 모델.
* **비유:** 위조지폐범(생성자)과 위조지폐를 감별하는 경찰(판별자)의 경쟁.
    * **생성자 (Generator):** 진짜 같은 가짜 데이터(위조지폐)를 생성.
    * **판별자 (Discriminator):** 입력된 데이터가 진짜(진짜 돈)인지 가짜(위조지폐)인지 판별.
* **궁극적 목적:** 실제 데이터의 분포에 가까운 데이터를 생성. (판별기가 진짜/가짜를 50:50으로 구분 못하게 되는 지점)
* **활용:** 이미지 생성, 영상 합성 등.

### (1) GAN 적용 사례

* **NVIDIA (2017):** 고해상도 가짜 인물/사물 이미지 생성 (Progressive Growing of GANs). 저해상도에서 고해상도로 점진적 학습.
* **워싱턴대 (2017):** 가짜 오바마 연설 영상. 음성에 맞는 입모양을 GAN으로 합성.
* **Facebook (2017):** 눈 감은 사진에 진짜 같은 가짜 눈을 합성 (Eye In-Painting, ExGAN).

### (2) GAN의 장단점

* **장점:** 진짜 같은 고품질의 가짜 데이터 생성 가능.
* **한계점 (단점):**
    * **학습 불안정 (Training instability):** 학습 과정에서 진동(Oscilation)이 발생하며 안정화가 어려움. (Generator와 Discriminator가 끝없는 숨바꼭질)
    * **Mode Collapsing:** 생성자가 판별자를 속이기 쉬운 하나의 결과물(Mode)만 편향적으로 생성하는 현상. (예: 숫자 생성 시 '1'만 계속 생성)

---

## 2-4. 확산 모델(Diffusion Model)

* **정의:** 입력 이미지에 노이즈(Noise)를 여러 단계에 걸쳐 추가(Forward)하고, 다시 여러 단계에 걸쳐 노이즈를 제거(Reverse)함으로써 원본과 유사한 이미지를 생성하는 모델.
* **목표:** (Forward → Reverse) 단계를 거친 '결과 이미지'를 '입력 이미지'의 확률 분포와 유사하게 만듦.

### (1) 순확산과 역확산

* **순확산 (Forward Diffusion Process):** 원본 데이터(깨끗한 이미지)에 점진적으로 노이즈를 추가하여, 최종적으로 완전한 무작위 노이즈로 변형시키는 과정.
* **역확산 (Reverse Diffusion Process):** 완전한 노이즈 데이터에서 시작하여, 순확산의 반대 방향으로 노이즈를 점진적으로 제거(예측)하며 원본 데이터를 복원(생성)하는 과정.

### (22) 확산 모델 예제


* AE, VAE, GAN, Diffusion Model은 데이터를 '새롭게 만들어내는' 생성 모델의 대표 예시.
    * **언어 모델 (LLM):** ChatGPT, Gemini 등 → **Transformer (Decoder-only)**
    * **이미지 생성:** DALL·E 3, Stable Diffusion → **Diffusion 기반 (주로 Latent Diffusion)**
    * **음성 생성:** VALL-E, Suno → **Transformer + VAE/Diffusion 혼합형**
    * **영상 생성:** Sora, Runway → **Diffusion 확장형 (Video Diffusion, 시공간 확산)**